In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 0. Test simple video feed

In [2]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 1. Make detections

In [2]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('data/50_ways_to_fall.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # use break if capturing a video file, continue if webcam

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # mediapipe requires RGB
        image.flags.writeable = False # helps manage memory

        # Make detections
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 2. Determining joints

<img src='https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png' style='height:300px'>

In [4]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('data/50_ways_to_fall.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # use break if capturing a video file, continue if webcam

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # mediapipe requires RGB
        image.flags.writeable = False # helps manage memory

        # Make detections
        results = pose.process(image)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            # print(landmarks)
        except:
            pass

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Utilize landmarks

## Calculate perceived width and height from all joints

In [5]:
def calculate_height_width(landmarks):
    '''
    Calculate perceived width and height by finding the range between extremes of x amd y coordinates respectively

    Input:
    landmarks - pose_landmarks field that contains the pose landmarks (returned from mp.solutions.pose.Pose.process)

    Returns:
    perceived height, perceived width
    '''
    joints_x, joints_y = [landmarks[i].x for i in range(0,33)], [landmarks[i].y for i in range(0,33)]
    min_x, min_y = min(joints_x), min(joints_y)
    max_x, max_y = max(joints_x), max(joints_y)
    perceived_width = max_x - min_x
    perceived_height = max_y - min_y
    return perceived_height, perceived_width
def annotate_height_width(image, height, width):
    cv2.putText(
        image, f'Height = {height}',
        tuple(np.multiply(0.02, image.shape[:2]).astype(int)), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0)
        )
    cv2.putText(
        image, f'Width = {width}',
        tuple(np.multiply(0.04, image.shape[:2]).astype(int)), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0)
        )

calculate_height_width(landmarks)

(0.1769704818725586, 0.28001585602760315)

In [6]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('data/50_ways_to_fall.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # use break if capturing a video file, continue if webcam

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # mediapipe requires RGB
        image.flags.writeable = False # helps manage memory

        # Make detections
        results = pose.process(image)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            height, width = calculate_height_width(landmarks)
            annotate_height_width(image, height, width)
        except:
            pass

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Calculate perceived width and height from an enclosing rectangle on the body

In [7]:
def get_enclosing_rectangle(landmarks):
    '''
    Get coordinates for a rectangle enclosing the body derived from extreme points of the body.
    - nose
    - left and right: shoulders, wrists, heels
    Input:
    landmarks - pose_landmarks field that contains the pose landmarks (returned from mp.solutions.pose.Pose.process)

    Returns:
    rectangle - ((left_corner_x, left_corner_y), (right_corner_x, right_corner_y), height, width) 
                x, y are from 0 - 1
    '''   
    extreme_joints_ix = [0,11,12,15,16,29,30]
    x = [landmarks[ix].x for ix in extreme_joints_ix]
    y = [landmarks[ix].y for ix in extreme_joints_ix]
    return (
        (min(x),max(y)),
        (max(x),min(y)),
        max(y)-min(y),
        max(x)-min(x))
def annotate_rectangle(image, rectangle):
    image_height, image_width = image.shape[0], image.shape[1]
    rectangle_lower_left_x = int(rectangle[0][0] * image_width)
    rectangle_lower_left_y = int(rectangle[0][1] * image_height)
    rectangle_upper_right_x = int(rectangle[1][0] * image_width)
    rectangle_upper_right_y = int(rectangle[1][1] * image_height)
    cv2.rectangle(
        image,
        (rectangle_lower_left_x, rectangle_lower_left_y),
        (rectangle_upper_right_x, rectangle_upper_right_y),
        (54, 89, 255), 5
        )
        
    rectangle_height = rectangle[2]
    rectangle_width = rectangle[3]
    cv2.putText(
        image, f'Rectangle Height = {rectangle_height}',
        (int(0.02*image_width), int(0.04*image_height)), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0)
        )
    cv2.putText(
        image, f'Rectangle Width = {rectangle_width}',
        (int(0.02*image_width), int(0.07*image_height)), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0)
        )   
get_enclosing_rectangle(landmarks)

((0.07834063470363617, 0.8107044696807861),
 (0.3077361583709717, 0.5618168711662292),
 0.24888759851455688,
 0.2293955236673355)

In [8]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('data/50_ways_to_fall.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # use break if capturing a video file, continue if webcam

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # mediapipe requires RGB
        image.flags.writeable = False # helps manage memory

        # Make detections
        results = pose.process(image)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            rectangle = get_enclosing_rectangle(landmarks)
            annotate_rectangle(image, rectangle)
        except:
            pass

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 4. Count falls

## All joints

In [9]:
def annotate_fall_count(image, height, width, fall_count):
    annotate_height_width(image, height, width)
    cv2.putText(
        image, f'Fall count = {fall_count}',
        tuple(np.multiply(0.06, image.shape[:2]).astype(int)), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0)
        )
def detect_fall(height_0, width_0, height_1, width_1):
    '''
    Detect a fall when 
    - width_1 > height_1 (current frame)
    and width_0 <= height_0 (previous frame)

    Params:
    height_0, width_0 - height and width from the previous frame
    height_1, width_1 - height and width from the current frame
    '''
    return width_1 > height_1 and width_0 <= height_0


In [10]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('data/50_ways_to_fall.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # init fall_count and height_0, width_0
    fall_count = 0
    height_0 = None
    width_0 = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # use break if capturing a video file, continue if webcam

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # mediapipe requires RGB
        image.flags.writeable = False # helps manage memory

        # Make detections
        results = pose.process(image)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            # current frame
            height_1, width_1 = calculate_height_width(landmarks)
            # detect_fall
            if height_0 and width_0:
                if detect_fall(height_0, width_0, height_1, width_1):
                    fall_count += 1
            height_0, width_0 = height_1, width_1
            annotate_fall_count(image, height_1, width_1, fall_count)
        except:
            pass

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Enclosing rectangle

In [11]:
def annotate_rectangle_fall_count(image, rectangle, fall_count):
    annotate_rectangle(image, rectangle)
    image_height, image_width = image.shape[0], image.shape[1]
    cv2.putText(
        image, f'Fall count = {fall_count}',
        (int(0.02*image_width), int(0.1*image_height)), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255)
        )
def detect_rectangle_fall(rectangle_0, rectangle_1):
    '''
    Detect a fall when 
    - width_1 > height_1 (current frame)
    and width_0 <= height_0 (previous frame)

    Params:
    rectangle_0, rectangle_1 - return val of get_enclosing_rectangle from consecutive frames
    '''
    # return width_1 > height_1 and width_0 <= height_0
    return rectangle_1[3] > rectangle_1[2] and rectangle_0[3] <= rectangle_0[2]
    


In [15]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('data/50_ways_to_fall.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # init fall_count and height_0, width_0
    fall_count = 0
    rectangle_0 = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # use break if capturing a video file, continue if webcam

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # mediapipe requires RGB
        image.flags.writeable = False # helps manage memory

        # Make detections
        results = pose.process(image)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            # current frame
            rectangle_1 = get_enclosing_rectangle(landmarks)
            # detect_fall
            if rectangle_0:
                if detect_rectangle_fall(rectangle_0, rectangle_1):
                    fall_count += 1
            rectangle_0 = rectangle_1
            annotate_rectangle(image, rectangle_1)
            annotate_rectangle_fall_count(image, rectangle_1, fall_count)
        except:
            pass

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()